In [1]:
# Restart dfs
!stop-dfs.sh
!start-dfs.sh


Stopping namenodes on [0.0.0.0]
Stopping datanodes
Stopping secondary namenodes [worker01]
Starting namenodes on [0.0.0.0]
Starting datanodes
Starting secondary namenodes [worker01]


In [2]:
# Check java services running
!jps


41209 Jps
40844 DataNode
41053 SecondaryNameNode
40734 NameNode


In [3]:
# Check Safe mode: should be OFF
!hdfs dfsadmin -safemode get


Safe mode is ON


In [4]:
# If Safe mode is ON --> execute hdfs dfsadmin -safemode leave
!hdfs dfsadmin -safemode leave


Safe mode is OFF


In [5]:
# Check hdfs home files
!hdfs dfs -ls


Found 8 items
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:55 commerce
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:31 e-commerce
drwxr-xr-x   - hduser supergroup          0 2024-10-10 21:11 output
-rw-r--r--   3 hduser supergroup      12331 2024-10-26 10:57 taxi_zone_lookup.csv
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:44 test.json
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:44 train.json
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:44 validate.json
drwxr-xr-x   - hduser supergroup          0 2024-11-05 09:20 warehouse


In [6]:
# Create hdfs directories
!hdfs dfs -mkdir -p \
    e-commerce/datasets \
    e-commerce/splits \
    e-commerce/models \
    e-commerce/outputs


In [7]:
!hdfs dfs -ls e-commerce/


Found 4 items
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:44 e-commerce/datasets
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:27 e-commerce/models
drwxr-xr-x   - hduser supergroup          0 2024-12-19 08:29 e-commerce/outputs
drwxr-xr-x   - hduser supergroup          0 2024-12-20 20:29 e-commerce/splits


In [8]:
# Download kaggle dataset into hdfs

!kaggle datasets download -d robertvici/indonesia-top-ecommerce-unicorn-tweets -p ~/kaggle-datasets
!unzip -o ~/kaggle-datasets/indonesia-top-ecommerce-unicorn-tweets.zip -d ~/kaggle-datasets
!hdfs dfs -put ~/kaggle-datasets/*.json e-commerce/datasets/

Dataset URL: https://www.kaggle.com/datasets/robertvici/indonesia-top-ecommerce-unicorn-tweets
License(s): copyright-authors
indonesia-top-ecommerce-unicorn-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /home/hduser/kaggle-datasets/indonesia-top-ecommerce-unicorn-tweets.zip
  inflating: /home/hduser/kaggle-datasets/ShopeeID.json  
  inflating: /home/hduser/kaggle-datasets/bliblidotcom.json  
  inflating: /home/hduser/kaggle-datasets/bukalapak.json  
  inflating: /home/hduser/kaggle-datasets/lazadaID.json  
  inflating: /home/hduser/kaggle-datasets/tokopedia.json  
put: `e-commerce/datasets/bliblidotcom.json': File exists
put: `e-commerce/datasets/bukalapak.json': File exists
put: `e-commerce/datasets/lazadaID.json': File exists
put: `e-commerce/datasets/ShopeeID.json': File exists
put: `e-commerce/datasets/tokopedia.json': File exists


In [9]:
# Show list of dataset files downloaded
!hdfs dfs -ls /user/hduser/e-commerce/datasets


Found 5 items
-rw-r--r--   3 hduser supergroup  233134634 2024-12-19 08:44 /user/hduser/e-commerce/datasets/ShopeeID.json
-rw-r--r--   3 hduser supergroup   56417505 2024-12-19 08:44 /user/hduser/e-commerce/datasets/bliblidotcom.json
-rw-r--r--   3 hduser supergroup   46806779 2024-12-19 08:44 /user/hduser/e-commerce/datasets/bukalapak.json
-rw-r--r--   3 hduser supergroup  118395100 2024-12-19 08:44 /user/hduser/e-commerce/datasets/lazadaID.json
-rw-r--r--   3 hduser supergroup   29228846 2024-12-19 08:44 /user/hduser/e-commerce/datasets/tokopedia.json


In [10]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lower, regexp_replace, count
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lit
import tensorflow as tf
import numpy as np
from keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


2024-12-21 14:38:03.220086: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 14:38:03.234053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734766683.250834   40258 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734766683.255496   40258 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 14:38:03.271890: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [11]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("E-Commerce Engagement Prediction ML") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 14:38:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
# Load datasets
blibli_df = spark.read.json('e-commerce/datasets/bliblidotcom.json')
bukalapak_df = spark.read.json('e-commerce/datasets/bukalapak.json')
lazadaID_df = spark.read.json('e-commerce/datasets/lazadaID.json')
shopeeID_df = spark.read.json('e-commerce/datasets/ShopeeID.json')
tokopedia_df = spark.read.json('e-commerce/datasets/tokopedia.json')

# Add a new column to identify the company source
blibli_df = blibli_df.withColumn('source', lit('blibli'))
bukalapak_df = bukalapak_df.withColumn('source', lit('bukalapak'))
lazadaID_df = lazadaID_df.withColumn('source', lit('lazadaID'))
shopeeID_df = shopeeID_df.withColumn('source', lit('shopeeID'))
tokopedia_df = tokopedia_df.withColumn('source', lit('tokopedia'))

# Merge datasets using union (axis=0 equivalent in Spark)
merged_df = blibli_df.union(bukalapak_df).union(lazadaID_df).union(shopeeID_df).union(tokopedia_df)

# Show merged data
merged_df.show()


24/12/21 14:38:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------------------+-------------+----------+---+--------------------+-------------------+-----------+--------------------+-----------------+----------+----+--------------------+-----+---------+-------------+--------------------+-------+------------+----------+--------------+------+--------+--------+----------+---------+---------+--------------------+--------------------+---------+-------+----------+------------+-----+
|cashtags|    conversation_id|   created_at|      date|geo|            hashtags|                 id|likes_count|                link|         mentions|      name|near|              photos|place|quote_url|replies_count|            reply_to|retweet|retweet_date|retweet_id|retweets_count|source|    time|timezone|trans_dest|trans_src|translate|               tweet|                urls|  user_id|user_rt|user_rt_id|    username|video|
+--------+-------------------+-------------+----------+---+--------------------+-------------------+-----------+--------------------+-

In [13]:
# DataFrame structure
print("Schema of the merged DataFrame:")
merged_df.printSchema()

Schema of the merged DataFrame:
root
 |-- cashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: long (nullable = true)
 |-- date: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- likes_count: long (nullable = true)
 |-- link: string (nullable = true)
 |-- mentions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- near: string (nullable = true)
 |-- photos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- place: string (nullable = true)
 |-- quote_url: string (nullable = true)
 |-- replies_count: long (nullable = true)
 |-- reply_to: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user_id: string (nullable = true)
 |    |    |-- userna

In [14]:
# Number of rows
row_count = merged_df.count()
print(f"Number of rows in the merged DataFrame: {row_count}")

Number of rows in the merged DataFrame: 541180


In [15]:
# Clean tweet text
def clean_text(text):
    return text.lower().replace("#", "").strip()

clean_text_udf = udf(clean_text, StringType())

# Apply text cleaning and create new features
data_cleaned = merged_df.withColumn("clean_tweet", clean_text_udf(col("tweet"))) \
                       .withColumn("engagement", col("replies_count") + col("retweets_count") + col("likes_count"))

# Select relevant features
selected_data = data_cleaned.select(
    col("clean_tweet").alias("text"),
    col("replies_count").alias("replies"),
    col("retweets_count").alias("retweets"),
    col("likes_count").alias("likes"),
    col("engagement").alias("target"),
    col("hashtags"),    
    col("source")
)

# Show processed data
selected_data.show()


+--------------------+-------+--------+-----+------+--------------------+------+
|                text|replies|retweets|likes|target|            hashtags|source|
+--------------------+-------+--------+-----+------+--------------------+------+
|sudah kepikiran m...|      0|       0|    1|     1| [#ramadanlebihbaik]|blibli|
|belanja pake kart...|      0|       0|    0|     0| [#ramadanlebihbaik]|blibli|
|udah seminggu ram...|      0|       0|    1|     1|[#ramadan, #ramad...|blibli|
|catet tanggal mai...|      0|       1|    1|     2|                  []|blibli|
|yuk! gabung di ng...|      1|       1|    1|     3|    [#ngobrolbareng]|blibli|
|udah bosen banget...|      1|       2|    1|     4|                  []|blibli|
|long weekend! bak...|      1|       1|    3|     5|       [#dirumahaja]|blibli|
|selamat hari buru...|      0|       0|    3|     3|       [#mayday2020]|blibli|
|cek di sini, ya h...|      0|       0|    0|     0|                  []|blibli|
|buang semua racun...|      

In [16]:
# Split dataset
train_data, validate_data, test_data = selected_data.randomSplit([0.7, 0.15, 0.15], seed=42)

# Save splits for later use
train_data.write.json("e-commerce/splits/train.json", mode="overwrite")
validate_data.write.json("commerce/splits/validate.json", mode="overwrite")
test_data.write.json("commerce/splits/test.json", mode="overwrite")

24/12/21 14:38:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [17]:
# check null values

merged_df.filter(
    (merged_df.likes_count.isNull()) | 
    (merged_df.replies_count.isNull()) | 
    (merged_df.retweets_count.isNull())
).show()


+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
|cashtags|conversation_id|created_at|date|geo|hashtags| id|likes_count|link|mentions|name|near|photos|place|quote_url|replies_count|reply_to|retweet|retweet_date|retweet_id|retweets_count|source|time|timezone|trans_dest|trans_src|translate|tweet|urls|user_id|user_rt|user_rt_id|username|video|
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+

In [18]:
# Change null value with 0 (if any)
merged_df = merged_df.fillna({"likes_count": 0, "replies_count": 0, "retweets_count": 0})


In [19]:
from pyspark.sql import functions as F

# Check negative value
merged_df.filter((F.col("likes_count") < 0) | (F.col("replies_count") < 0) | (F.col("retweets_count") < 0)).show()


+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
|cashtags|conversation_id|created_at|date|geo|hashtags| id|likes_count|link|mentions|name|near|photos|place|quote_url|replies_count|reply_to|retweet|retweet_date|retweet_id|retweets_count|source|time|timezone|trans_dest|trans_src|translate|tweet|urls|user_id|user_rt|user_rt_id|username|video|
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+

In [20]:
# Change negative value with 0 (if any)
for col in ["likes_count", "replies_count", "retweets_count"]:
    merged_df = merged_df.withColumn(col, F.when(F.col(col) < 0, 0).otherwise(F.col(col)))
    

In [21]:
# Matching target engagement definitions in Spark DataFrame
blibli_df = blibli_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
bukalapak_df = bukalapak_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
lazadaID_df = lazadaID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
shopeeID_df = shopeeID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
tokopedia_df = tokopedia_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))


In [22]:
# Load train data (convert Spark DataFrame to Pandas)
train_df = train_data.toPandas()

# Tokenize and vectorize text (fit on original text, not the padded sequences)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df["text"])  # Fit tokenizer on the raw text data

# Convert texts to sequences
X_train = tokenizer.texts_to_sequences(train_df["text"])

# Pad the sequences to ensure uniform length
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')

y_train = np.array(train_df["target"])


In [23]:
# Define a simple Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])

In [24]:
# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

2024-12-21 14:38:40.622870: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [25]:
vocab_size = 42500  # Customize with your tokenizer
embedding_dim = 128

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)


In [26]:
# Example: Tokenize and pad the input text
max_vocab_size = 5000
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(train_df["text"])  # 'train_df["text"]' should be a list of strings

# Example of saving the tokenizer
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

X_train_sequences = tokenizer.texts_to_sequences(train_df["text"])
X_train = pad_sequences(X_train_sequences, padding='post')

# Ensure y_train is in the correct format (e.g., a numpy array)
y_train = np.array(train_df["target"])  # Adjust this based on your target column

# Train the model !!!
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 773637.2500 - mae: 30.9336
Epoch 2/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 710051.8125 - mae: 29.2274
Epoch 3/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 617176.1875 - mae: 27.7469
Epoch 4/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 759199.1875 - mae: 25.9931
Epoch 5/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 498374.6562 - mae: 24.8291
Epoch 6/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 590415.5625 - mae: 27.3335
Epoch 7/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 608493.3750 - mae: 29.8498
Epoch 8/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 1337413.2500 - mae: 33.3928
Epoch 9/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 394809.8438 - mae: 27.5972
Epoch 10/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 236905.9531 - mae: 26.5278


In [27]:
# Save the model with a valid file extension in local server
model.save("e-commerce-engagement_model_notebook.keras")  # For the native Keras format

In [28]:
# Export to save model
model.export("saved_mode_notebook/1")

INFO:tensorflow:Assets written to: saved_mode_notebook/1/assets


INFO:tensorflow:Assets written to: saved_mode_notebook/1/assets


Saved artifact at 'saved_mode_notebook/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140354983187408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140354983187984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140354983188752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140354983188176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140354983190096: TensorSpec(shape=(), dtype=tf.resource, name=None)


## API Gateway with Flask/FastAPI

Create endpoint to call TensorFlow Serving API:

1. pip install flask requests

2. Create predict_model.py
   
   ```
   from flask import Flask, request, jsonify
    import pickle
    import numpy as np
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.models import load_model
    
    # Initialize Flask app
    app = Flask(__name__)
    
    # Load tokenizer and model
    with open('tokenizer.pkl', 'rb') as file:
        tokenizer = pickle.load(file)
    
    model = load_model('engagement_model.keras')
    
    # Set a maximum sequence length (should match what was used during training)
    MAX_SEQUENCE_LENGTH = 20
    
    @app.route('/predict', methods=['POST'])
    def predict():
        data = request.get_json()
        input_text = data.get('text', '')
    
        if not input_text:
            return jsonify({'error': 'No text provided'}), 400
    
        try:
            # Convert text to sequences
            sequences = tokenizer.texts_to_sequences([input_text])
            
            # Pad the sequences to ensure consistent input shape
            padded_input = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    
            # Convert to NumPy array (Keras requires this)
            processed_input = np.array(padded_input)
    
            # Perform prediction
            prediction = model.predict(processed_input)
    
            # Return prediction
            return jsonify({'prediction': prediction.tolist()})
    
        except Exception as e:
            return jsonify({'error': str(e)}), 500
    
    
   if __name__ == "__main__":
        app.run(host='0.0.0.0', port=5000)

    ```

3. Run API

    ```
    python predict_model.py
   
    ```
  